In [3]:
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

import pyperclip
import time
import datetime

def top_to_bottom(browser, i = None):
    start = time.time()
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(2.5)

        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            time.sleep(1)
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                return
        last_height = new_height  
        if i:
            if (time.time() - start) > i:
                break
                
def print_csv(f_name, name, contact, city):
    try:
        df = pd.DataFrame({"이름":name,
                           "연락처":contact,
                           "지역":city})
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df.isnull().sum()
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df['출생연도'] = int_values
        df.columns = ['이름', '연락처', '지역', '출생연도']
        df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')     
        
        if len(df[df['연락처'].str.contains('\+82')]) >= 1:
            df['연락처'] = df['연락처'].str.split('\+82 ').str.get(1).str[0:12].fillna('없음')
            df.index[df['연락처'].str.contains('1')]
            phone_num = df.index[df['연락처'].str.contains('1')]
            df.loc[phone_num, '연락처'] = "0" + df.loc[phone_num, '연락처']
        else:
            df['연락처'] = '없음'
        df.to_csv(f"{f_name}.csv", encoding='UTF-8')
    except Exception as ex:
        print(f'{len(name)} {len(contact)} {len(city)}')
        df = pd.DataFrame({"이름":name,
                           "연락처":contact,
                           "지역":city})

#open browser with secret mode
options = webdriver.ChromeOptions()
options.add_argument("--proxy-server=socks5://127.0.0.1:9150")
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--no-sandbox')
options.add_argument('no-sandox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized')
options.add_argument('incognito')

service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)

#open facebook
browser.get("https://www.facebook.com/")
time.sleep(2)

#log info, search keyword
USER = "miraemakers@naver.com"
PWD = "445566mmk@"

#login
elem_id = browser.find_element("id", "email")
pyperclip.copy(USER)
elem_id.send_keys(Keys.CONTROL, "v")
time.sleep(2)

elem_pw = browser.find_element("id", "pass")
pyperclip.copy(PWD)
elem_pw.send_keys(Keys.CONTROL, "v")
elem_pw.send_keys("\n")
time.sleep(3)

final_href = ['https://www.facebook.com/groups/178047442249023/members',
              'https://www.facebook.com/groups/171371239873412/members',
              'https://www.facebook.com/groups/107853536013311/members',
              'https://www.facebook.com/groups/915098498571012/members',
              'https://www.facebook.com/groups/497296441593484/members',
              'https://www.facebook.com/groups/266566810045702/members'
             ]

df_name, df_contact, df_city = [], [], []

    #loop with groups
for i in final_href:
    #open the group page
    browser.get(i)

    #except block alert
    try:
        elem_block = browser.find_element("xpath", "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div")
        elem_block.click()
    except:
        pass
    try:
        elem_block = browser.find_element('xpath', '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
        elem_block.click()
    except:
        pass
    time.sleep(1)

    # scroll to bottom
    member = browser.find_element('xpath', '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div/div[2]/div/div/div[4]/div/div/div/div/div/div/div/div/div/div/div[1]/div/div/div/div/div[1]/h2/span/span/span/strong')
    member = member.text[4:]
    while True:
        top_to_bottom(browser)
        time.sleep(1.5)

        #get pofile links
        soup_profile = bs(browser.page_source, "html.parser")
        h_list = soup_profile.find_all('h1')
        top_elements = soup_profile.find_all(class_="x1lq5wgf xgqcy7u x30kzoy x9jhf4c x1lliihq")

        #get profile links
        profile_list = []
        profile_href = []
        for j in top_elements:
            profile_list.append(j.find("a").attrs['href'])

        if len(profile_list) >= (int(member)*0.9):
            break

    for j in profile_list:
        profile_href.append('https://www.facebook.com/profile.php?id='+j[j.index('user/')+5:-1])
    
    #get group name
    for h in h_list:
        if h.text == '알림': continue
        elif h.text == '검색결과': continue
        else: g_name = h.text
    print(g_name)

    #save elements
    name, contact, city = [], [], []
    error = []
    url = 'php?id='
    print(len(profile_href))

    #loop with profiles
    for j in profile_href:
        try:
            print(profile_href.index(j))
            #open the profile page
            browser.get(j)
            #is profile link with id or numbers
            temp = browser.current_url
            if url in temp:
                link = j + '&sk='
            else:
                link = temp[:temp.index('?')]
            #open 
            browser.get(link+"about_contact_and_basic_info")
            soup_info = bs(browser.page_source, "html.parser")
            soup_name = soup_info.find(class_="x78zum5 x15sbx0n x5oxk1f x1jxijyj xym1h4x xuy2c7u x1ltux0g xc9uqle")
            soup_info_contact = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
            t_cont = soup_info_contact.text
            if '+82 ' in t_cont:
                contact.append(t_cont)
                name.append(soup_name.find("h1").text)
                browser.get(link+"about_places")
                soup_info = bs(browser.page_source, "html.parser")
                soup_info_city = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                city.append(soup_info_city.text)
            else: continue

        except:
            l_name = len(name)
            l_cont = len(contact)
            l_city = len(city)
            print(browser.current_url)
            error.append(browser.current_url)
            if l_name > l_cont:
                contact.append('이전 거주지표시할 장소 없음')
                try:
                    browser.get(link+"about_places")
                    soup_info = bs(browser.page_source, "html.parser")
                    soup_info_ct = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                    city.append(soup_info_ct.text)
                except:
                    city.append('')
            elif l_cont > l_city:
                city.append('')
            else: 
                continue

    # get real time
    timestamp = datetime.datetime.now()
    t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
    t_filename = f'{g_name}_{t_stamp}'

    if len(name) == 0:
#         group_name.append(g_name)
#         f_name.append(' ')
        print(f'{g_name}: nothing in there')
        continue
    else:
        df_name.append(name)
        df_contact.append(contact)
        df_city.append(city)
        print_csv(t_filename, name, contact, city)
        print(f'{g_name}: {t_filename}')
#         group_name.append(g_name)
#         f_name.append(t_filename)


print_csv('군포 총합', df_name, df_contact, df_city)

timestamp = datetime.datetime.now()
t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
t_filename = f'{g_name}_filelist_{t_stamp}'


# df = pd.DataFrame({"그룹 이름":group_name,
#                    "파일명":f_name})
# df.to_csv(f"{t_filename}.csv", encoding='UTF-8')


군포,산본,안양,평촌,의왕,과천 이야기
426
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
https://www.facebook.com/yeonseshintong/about_places
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
0
251
252
253
254
255
256
257
2

NameError: name 'df_conctact' is not defined

In [9]:
        print_csv(t_filename, name, contact, city)
        print(f'{g_name}: {t_filename}')

군포,산본,안양,평촌,의왕,과천 이야기: 군포,산본,안양,평촌,의왕,과천 이야기_2023-06-27_154948
